In [2]:
import pandas as pd
import plotly.express as px

In [3]:
df = pd.read_csv("../data/bandwidths.csv", parse_dates=["Timestamp"])
df

,Timestamp,Central America,Africa,Middle East,Oceania,South America,Russia,Asia,Europe,North America
0,2024-06-12 16:50:00,63,124,932,176,1152,1948,6941,7953,4087
1,2024-06-12 17:00:00,62,122,932,176,1152,1902,6444,7941,4087
2,2024-06-12 17:10:00,63,124,932,177,1162,1931,6444,7953,4130
3,2024-06-12 17:20:00,62,122,943,180,1175,1931,6025,8061,4206
4,2024-06-12 17:30:00,61,120,943,180,1175,1950,5663,8061,4219
...,...,...,...,...,...,...,...,...,...,...
1498,2024-06-23 02:30:00,105,46,359,472,1867,369,6000,1277,6730
1499,2024-06-23 02:40:00,105,44,342,477,1849,361,6244,1231,6724
1500,2024-06-23 02:50:00,104,44,342,488,1841,353,6590,1185,6689
1501,2024-06-23 03:00:00,100,42,330,494,1814,353,6785,1127,6552


In [4]:
df_melted = df.melt(
    id_vars=["Timestamp"], 
    value_vars=df.columns.to_list()[1:], 
    var_name="Region", 
    value_name="Bandwidth",
)
df_melted

,Timestamp,Region,Bandwidth
0,2024-06-12 16:50:00,Central America,63
1,2024-06-12 17:00:00,Central America,62
2,2024-06-12 17:10:00,Central America,63
3,2024-06-12 17:20:00,Central America,62
4,2024-06-12 17:30:00,Central America,61
...,...,...,...
13522,2024-06-23 02:30:00,North America,6730
13523,2024-06-23 02:40:00,North America,6724
13524,2024-06-23 02:50:00,North America,6689
13525,2024-06-23 03:00:00,North America,6552


In [5]:
stacked = px.area(
    data_frame=df_melted, 
    x="Timestamp", 
    y="Bandwidth", 
    color="Region", 
    line_group="Region",
    labels={"Timestamp": "Date", "Bandwidth": "Bandwidth (Gbps)"},
    title="Stacked Area Chart of Download Bandwidth Usage on Steam",
    color_discrete_sequence=px.colors.sequential.Plasma_r,
    width=1080, 
    height=720,
)
stacked.show()

In [6]:
faceted = px.line(
    data_frame=df_melted,
    x="Timestamp",
    y="Bandwidth",
    facet_col="Region",
    facet_col_wrap=1,
    facet_row_spacing=0.03,
    labels={"Timestamp": "Date"},
    title="Steam Download Bandwidth Used by Region",
    width=1080,
    height=1200,
).update_yaxes(matches=None)

faceted.for_each_yaxis(lambda y: y.update({"title": ""}))
faceted.add_annotation(
    showarrow=False,
    xanchor='center',
    xref='paper', 
    x=-0.05, 
    yanchor='middle',
    yref='paper',
    y=0.5,
    textangle=270,
    text='Bandwidth (Gbps)'
)
faceted.for_each_annotation(lambda x: x.update(text=x.text.split("=")[-1]))

faceted.show()

In [7]:
df_agg = pd.DataFrame()
df_agg["Timestamp"] = df["Timestamp"].copy()
df_agg["Total Bandwidth"] = df.sum(axis=1, numeric_only=True)

mean = df_agg["Total Bandwidth"].mean()
std = df_agg["Total Bandwidth"].std()

total = px.line(
    data_frame=df_agg,
    x="Timestamp",
    y="Total Bandwidth",
    labels={"Timestamp": "Date", "Total Bandwidth": "Bandwidth (Gbps)"},
    title="Total Bandwidth Across All Regions on Steam",
    width=1080,
    height=720,
).update_traces(
    line_color="black", 
    line_width=3
).update_layout(
    # yaxis_range=[mean - 2 * std, mean + 2.5 * std]
)
total.show()

In [8]:
df2 = pd.read_csv("../data/support_requests.csv", parse_dates=["Timestamp"])
df2

,Timestamp,Submitted,Waiting for response
0,2023-12-18 07:00:00,253515,21443
1,2023-12-19 07:00:00,265505,18182
2,2023-12-20 07:00:00,295719,18770
3,2023-12-21 07:00:00,399779,16506
4,2023-12-22 07:00:00,640044,18188
...,...,...,...
182,2024-06-17 07:00:00,235167,18790
183,2024-06-18 07:00:00,250322,17910
184,2024-06-19 07:00:00,249272,19270
185,2024-06-20 07:00:00,269529,14535


In [9]:
df2_melted = df2.melt(
    id_vars=["Timestamp"], 
    value_vars=df2.columns.to_list()[1:], 
    var_name="Request Status", 
    value_name="Support Requests",
)
df2_melted

,Timestamp,Request Status,Support Requests
0,2023-12-18 07:00:00,Submitted,253515
1,2023-12-19 07:00:00,Submitted,265505
2,2023-12-20 07:00:00,Submitted,295719
3,2023-12-21 07:00:00,Submitted,399779
4,2023-12-22 07:00:00,Submitted,640044
...,...,...,...
369,2024-06-17 07:00:00,Waiting for response,18790
370,2024-06-18 07:00:00,Waiting for response,17910
371,2024-06-19 07:00:00,Waiting for response,19270
372,2024-06-20 07:00:00,Waiting for response,14535


In [11]:
fig = px.line(
    data_frame=df2_melted,
    x="Timestamp",
    y="Support Requests",
    color="Request Status",
    labels={"Timestamp": "Date", "Support Requests": "Number of Support Requests"},
    title="Support Requests Over Time",
    width=1080,
    height=720,
)
fig.show()